In [15]:
import sys
sys.path.append('..')
import numpy as np
from util import load_pkl
from scipy.optimize import minimize

In [20]:
data_dir = f'../processed_data_128'
S = load_pkl(f'{data_dir}/S.pkl')
mu = np.load(f'{data_dir}/mu.npy')


with open(f'{data_dir}/final_tickers.txt', 'r') as f:
    final_tickers = f.read().splitlines()

In [17]:
mu

array([ 1.77767519e-02,  3.89703236e-02,  1.09361695e-02,  4.17355103e-02,
        7.93483110e-03,  4.26573644e-02,  4.72843777e-02, -3.43106038e-02,
       -3.58071533e-02,  1.62124722e-02, -4.82695367e-02, -1.28985124e-02,
       -3.53506266e-02,  3.36149605e-02,  3.82900215e-02,  1.49698341e-02,
        5.86658774e-02,  2.15329073e-03,  7.52322078e-03, -5.24167628e-03,
        5.36099552e-02,  3.85997229e-02,  6.72739885e-02,  4.37418195e-02,
       -2.30910508e-02,  6.33018198e-03,  1.04090436e-01,  2.25799922e-02,
        3.76810255e-02,  9.11553348e-03,  1.46117047e-02,  4.26047870e-02,
        9.47946085e-02,  2.60837537e-03,  4.71440432e-03,  6.39323664e-03,
        3.64967570e-02,  6.19768128e-02,  5.65077284e-02,  3.23950729e-02,
        3.84561864e-02,  1.41015766e-02,  7.27117008e-02,  1.51588797e-02,
        4.19549276e-03,  8.91774448e-02,  2.32596802e-02,  4.74536878e-02,
        2.53059316e-02, -1.80027938e-02,  4.20643473e-02,  4.02527943e-02,
        4.43396573e-02,  

In [233]:
def portfolio_volatility_log_return(weights, covariance):
    return np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

def portfolio_log_return(weights, returns, allow_short=False):
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)

def portfolio_volatility(weights, covariance_log_returns):
    covariance_returns = np.exp(covariance_log_returns) - 1
    return np.sqrt(np.dot(weights.T, np.dot(covariance_returns, weights)))

def portfolio_return(weights, log_returns, allow_short=False):
    returns = np.exp(log_returns) - 1

    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)

def min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short=False):
    portfolio_ret = portfolio_log_return(weights, returns, allow_short)
    portfolio_vol = portfolio_volatility_log_return(weights, covariance)
    sharpe_ratio = (portfolio_ret - risk_free_rate) / portfolio_vol
    #return -sharpe_ratio # Negate Sharpe ratio because we minimize the function
    return - (portfolio_ret - 2 * portfolio_vol)



def optimize_portfolio(returns, covariance, risk_free_rate, allow_short=False):
    num_assets = len(returns)
    args = (returns, covariance, risk_free_rate)

    # Define constraints
    def constraint_sum(weights):
        return np.sum(weights) - 1
    
    constraints = [{'type': 'eq', 'fun': constraint_sum}]

    bounds = tuple((0.0, 0.20) for _ in range(num_assets))

    # Perform optimization
    def objective(weights):
        return min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short)
    
    iteration = [0]  # mutable container to store iteration count
    def callback(weights):
        iteration[0] += 1
        
        print(f"Iteration: {iteration[0]}, value: {objective(weights)}")

    # Initial guess (equal weights)
    initial_guess = num_assets * [1. / num_assets]

    # Perform optimization
    result = minimize(objective, initial_guess, 
                      method='SLSQP', bounds=bounds, constraints=constraints, callback=callback, options={'maxiter': 100})

    return result

In [234]:
INTEREST_RATE = 0.0497    # Current interest rate accessible for USD
ANNUAL_TRADING_DAYS = 252
MAX_RISK = 0.08


riskfree_log_return = np.log(1 + INTEREST_RATE) * 128 / ANNUAL_TRADING_DAYS
raw_weights = optimize_portfolio(mu, S, riskfree_log_return)

Iteration: 1, value: 0.13675634275506332
Iteration: 2, value: 0.03209919566061609
Iteration: 3, value: 0.2047335900869824
Iteration: 4, value: 0.182763089551462
Iteration: 5, value: 0.010119411840546752
Iteration: 6, value: 0.006134812435341891
Iteration: 7, value: -0.04466354643395494
Iteration: 8, value: -0.04917170898055806
Iteration: 9, value: -0.04868734367003767
Iteration: 10, value: -0.052112958936091205
Iteration: 11, value: -0.052346532533881704
Iteration: 12, value: -0.05249123672485646
Iteration: 13, value: -0.05255509813328105
Iteration: 14, value: -0.05259199319080869
Iteration: 15, value: -0.0526005920670551
Iteration: 16, value: -0.052604410363099374
Iteration: 17, value: -0.052606774562389255
Iteration: 18, value: -0.052606774562389255


In [235]:
raw_weights

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.052606774562389255
       x: [ 4.833e-16  0.000e+00 ...  1.161e-16  3.931e-16]
     nit: 18
     jac: [ 6.837e-02  9.170e-02 ...  2.657e-02  1.185e-01]
    nfev: 11129
    njev: 18

In [236]:
raw_weights_x = raw_weights.x

In [237]:
np.sum(raw_weights_x)

1.000000000000082

In [238]:
for index, ticker_name in enumerate(final_tickers):
    weight = raw_weights_x[index]
    if weight > 1e-6:
        print(f'index: {index} {ticker_name}: weight {weight} exp profit: {mu[index]}, variance: {S[ticker_name][ticker_name]}')

period = 128
print(f'expected return in {period} trading days: {portfolio_return(raw_weights_x, mu)}')
print(f'volatility of the return in {period} trading days: {portfolio_volatility(raw_weights_x, S)}')


index: 115 STAN.L: weight 0.06927842128841591 exp profit: 0.10238089417476481, variance: 0.04586061437233113
index: 158 AMGN: weight 0.07396852551768242 exp profit: 0.09861566943664779, variance: 0.017899267574992047
index: 189 BIIB: weight 0.042956047678477854 exp profit: 0.1988882007703651, variance: 0.07774654204843033
index: 249 COST: weight 0.035513428126730154 exp profit: 0.08356320316462489, variance: 0.02061973562158828
index: 250 CTRA: weight 0.05629593472364465 exp profit: 0.07433539701457556, variance: 0.0484500600172913
index: 283 LLY: weight 0.03914166003838209 exp profit: 0.11654037549544001, variance: 0.032870287324877165
index: 328 GILD: weight 0.1454579145864326 exp profit: 0.15744309058179476, variance: 0.026643804311949626
index: 341 HOLX: weight 0.05709098177517926 exp profit: 0.19164896779780063, variance: 0.03358620853875189
index: 369 JBL: weight 0.005640400607754833 exp profit: 0.19031114608892788, variance: 0.05287376579817663
index: 370 JKHY: weight 0.15277377

In [239]:
raw_weights_2 = optimize_portfolio(mu, S, riskfree_log_return, allow_short=True)
raw_weights_2_x = raw_weights_2.x

Iteration: 1, value: 0.1367563423042089
Iteration: 2, value: 0.03209919641236683
Iteration: 3, value: 0.20473358881393505
Iteration: 4, value: 0.19952818560087798
Iteration: 5, value: -0.002190249585423368
Iteration: 6, value: -0.04829069908752448
Iteration: 7, value: -0.05040468763414131
Iteration: 8, value: -0.04754132767329883
Iteration: 9, value: -0.05082016466109278
Iteration: 10, value: -0.05178309568144489
Iteration: 11, value: -0.052279730116660775
Iteration: 12, value: -0.05248179852138063
Iteration: 13, value: -0.052545873494688186
Iteration: 14, value: -0.0525883365871162
Iteration: 15, value: -0.052600491317548057
Iteration: 16, value: -0.05260395865504357
Iteration: 17, value: -0.05260647802896956
Iteration: 18, value: -0.05260647802896956


In [240]:
raw_weights_2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.05260647802896956
       x: [ 4.244e-16  0.000e+00 ...  0.000e+00  0.000e+00]
     nit: 18
     jac: [ 6.831e-02  9.132e-02 ...  2.669e-02  1.186e-01]
    nfev: 11130
    njev: 18

In [241]:
raw_weights_2_x

array([4.24429322e-16, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.65818772e-16,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.30363597e-17,
       1.41593073e-17, 0.00000000e+00, 0.00000000e+00, 9.17079943e-16,
       5.67361217e-16, 0.00000000e+00, 1.53176253e-17, 2.59083322e-16,
       2.22817702e-16, 4.27323341e-16, 0.00000000e+00, 9.67946787e-17,
       2.09405151e-16, 0.00000000e+00, 0.00000000e+00, 4.13292720e-16,
       9.50786903e-17, 4.87668810e-16, 0.00000000e+00, 1.79730103e-18,
       0.00000000e+00, 5.99865300e-17, 1.24492919e-16, 2.23011823e-16,
       3.90008417e-16, 2.47891453e-16, 6.32413044e-17, 9.15620402e-17,
       0.00000000e+00, 5.72276797e-16, 0.00000000e+00, 2.33671252e-18,
       1.77301921e-16, 2.51641190e-17, 3.62672193e-16, 1.35196618e-16,
       0.00000000e+00, 1.30313980e-16, 0.00000000e+00, 0.00000000e+00,
       6.14700673e-16, 8.79974803e-17, 2.41751515e-16, 1.47680889e-15,
      

In [242]:
np.sum(raw_weights_2_x)

1.0000000000000666

In [243]:

for index, ticker_name in enumerate(final_tickers):
    weight = raw_weights_2_x[index]
    if abs(weight) > 1e-2:
        print(f'index: {index} {ticker_name}: weight {weight} exp profit: {mu[index]}, variance: {S[ticker_name][ticker_name]}')

period = 128
print(f'expected return in {period} trading days: {portfolio_return(raw_weights_2_x, mu)}')
print(f'volatility of the return in {period} trading days: {portfolio_volatility(raw_weights_2_x, S)}')

index: 115 STAN.L: weight 0.06940932033704353 exp profit: 0.10238089417476481, variance: 0.04586061437233113
index: 158 AMGN: weight 0.07431243214457392 exp profit: 0.09861566943664779, variance: 0.017899267574992047
index: 189 BIIB: weight 0.042900139778976665 exp profit: 0.1988882007703651, variance: 0.07774654204843033
index: 249 COST: weight 0.03558101407191769 exp profit: 0.08356320316462489, variance: 0.02061973562158828
index: 250 CTRA: weight 0.05635014005671591 exp profit: 0.07433539701457556, variance: 0.0484500600172913
index: 283 LLY: weight 0.03948522028075085 exp profit: 0.11654037549544001, variance: 0.032870287324877165
index: 328 GILD: weight 0.1454264331821711 exp profit: 0.15744309058179476, variance: 0.026643804311949626
index: 341 HOLX: weight 0.05758084586182881 exp profit: 0.19164896779780063, variance: 0.03358620853875189
index: 370 JKHY: weight 0.1526600094065125 exp profit: 0.13122678600033347, variance: 0.011048992099250242
index: 410 MCK: weight 0.0723219528

In [232]:

def adjust_weights(weights, threshold=0.01, tolerance=1e-6):
    weights = np.array(weights)
    
    # Ensure the sum of absolute values of weights is 1
    abs_sum = np.sum(np.abs(weights))
    if abs_sum != 1:
        weights = weights / abs_sum

    run = 0
    while True:
        print(run)
        run += 1
        # Identify weights with absolute values below the threshold
        below_threshold = np.abs(weights) < threshold
        if not np.any(below_threshold):
            break

        # Find the minimal non-zero weight among those that are below the threshold
        invalid_weights = (np.abs(weights) < threshold) & (np.abs(weights) > tolerance)
        if np.any(invalid_weights):
            min_nonzero_weight = np.min(np.abs(weights[invalid_weights]))
            min_index = np.where(np.abs(weights) == min_nonzero_weight)[0][0]
        else:
            break


        # Set the minimal weight to zero
        min_value = weights[min_index]
        weights[min_index] = 0

        # Compute the deficit or surplus
        deficit = np.abs(min_value)

        # Spread this deficit or surplus equally among the remaining stocks (i.e., the ones with abs(weights) >= threshold)
        valid_weights = np.abs(weights) >= np.abs(min_value)
        adjustment = deficit / np.sum(valid_weights)
        
        # Adjust only the valid weights
        weights[valid_weights] += np.sign(weights[valid_weights]) * adjustment
        abs_sum = np.sum(np.abs(weights))
        print(abs_sum)
    return weights

In [67]:
adjusted_weights = adjust_weights(raw_weights_2)

0
1.0
1
1.0
2
0.9999999999999999
3
0.9999999999999999
4
0.9999999999999998
5
0.9999999999999999
6
0.9999999999999998
7
0.9999999999999999
8
0.9999999999999999
9
0.9999999999999998
10
1.0
11
0.9999999999999999
12
0.9999999999999999
13
0.9999999999999998
14
0.9999999999999999
15
0.9999999999999998
16
0.9999999999999999
17
0.9999999999999998
18
0.9999999999999997
19
0.9999999999999998
20
0.9999999999999998
21
0.9999999999999998
22
0.9999999999999998
23
0.9999999999999997
24
0.9999999999999998
25
0.9999999999999999
26
0.9999999999999999
27
0.9999999999999999
28
0.9999999999999999
29
0.9999999999999998
30
0.9999999999999998
31
1.0
32
0.9999999999999999
33
0.9999999999999998
34
0.9999999999999998
35
0.9999999999999998
36
0.9999999999999998
37
0.9999999999999999
38
0.9999999999999998
39
0.9999999999999999
40
0.9999999999999998
41
0.9999999999999998
42
0.9999999999999998
43
0.9999999999999999
44
0.9999999999999998
45
1.0
46
1.0
47
0.9999999999999998
48
1.0
49
0.9999999999999998
50
0.9999999999

In [69]:
for index, ticker_name in enumerate(final_tickers):
    weight = adjusted_weights[index]
    if weight != 0:
        print(f'index: {index} {ticker_name}: weight {weight} exp profit: {mu[index]}, variance: {S[ticker_name][ticker_name]}')

period = 128
print(f'expected return in {period} trading days: {portfolio_return(adjusted_weights, mu)}')
print(f'volatility of the return in {period} trading days: {portfolio_volatility(adjusted_weights, S)}')

index: 4 BAYN.DE: weight -0.010198953037516107 exp profit: 0.007934831103408728, variance: 0.05636878735389428
index: 7 BNR.DE: weight -0.01101896111033495 exp profit: -0.03431060380591024, variance: 0.045209597037622236
index: 12 DHL.DE: weight -0.010134905612508772 exp profit: -0.03535062656654784, variance: 0.06701840444315421
index: 24 PAH3.DE: weight -0.010111352816098186 exp profit: -0.023091050822275003, variance: 0.06473236879799919
index: 32 VOW3.DE: weight 0.012337935872867533 exp profit: 0.09479460852776339, variance: 0.05621442528413051
index: 37 ANTO.L: weight 0.010206852045053118 exp profit: 0.061976812827905146, variance: 0.07287534278567234
index: 53 CCH.L: weight 0.011355477232114391 exp profit: 0.07659219244254027, variance: 0.06142059283075459
index: 62 FCIT.L: weight 0.011194642645061786 exp profit: 0.07376576718726192, variance: 0.01737241474577895
index: 64 FRAS.L: weight 0.01316192934588307 exp profit: 0.21625988767808835, variance: 0.08884769741107183
index: 66 